In [1]:
search_for = 136
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0887455940246582
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 1048119951
type: [1, 1, 1, 1, 136] 136
cases of this type: 342102016
100000 35.71925464911737
200000 61.14208040888702
300000 55.577985495532715
400000 55.06335998696408
500000 57.31452719751752
600000 70.13305216504664
700000 82.0524780490313
800000 83.36927995054504
900000 82.70559865788373
1000000 88.32923757904273
1100000 100.32457707355908
1200000 99.92374945780128
1300000 31.423785649007034
1400000 89.15596514558801
1500000 99.00718778825909
1600000 100.21897042929893
1700000 108.08464628839081
1800000 116.9863382232757
1900000 110.38964614448666
2000000 109.25301086813135
2100000 103.19711209270591
2200000 104.96944284480637
2300000 119.14067583529162
2400000 108.6056056684248
2500000 92.37651731098082
2600000 36.01296139752725
2700000 101.37560974930551
2800000 97.43688650303285
2900000 97.67844513090391
3000000 109.35424

29300000 100.63134498692285
29400000 105.99345101737639
29500000 102.70147060848103
29600000 104.0059198944184
29700000 108.81744828157014
29800000 103.87727490658558
29900000 104.64860494272983
30000000 65.27852500091777
30100000 91.62763180389591
30200000 110.35001220364582
30300000 111.94717861619175
30400000 90.0938728786038
30500000 65.57271236223751
30600000 111.68086203246929
30700000 109.04126413618167
30800000 114.08185020647379
30900000 110.61347558127883
31000000 115.32411453281013
31100000 110.9968226935302
31200000 106.39707018063004
31300000 109.35542655788589
31400000 106.60608205532385
31500000 22.15700153985203
31600000 115.79589778827439
31700000 108.49800684991348
31800000 108.70867335615524
31900000 112.42083505843803
32000000 108.36880867063911
32100000 110.37986569674479
32200000 110.5978517732675
32300000 112.69103432516114
32400000 111.26603676898617
32500000 63.18523786409679
32600000 89.62089127706795
32700000 123.84484601644252
32800000 109.13409602787361
329

59000000 92.21931598267903
59100000 74.90779626838899
59200000 42.791752192464905
59300000 94.42473665226693
59400000 92.15057688012548
59500000 92.58191703453578
59600000 97.82421678750623
59700000 102.02894054177804
59800000 94.24141985953317
59900000 100.17475774889992
60000000 35.42542121156613
60100000 112.31806331262442
60200000 103.65706904337905
60300000 100.40947969213424
60400000 108.572982988173
60500000 107.3591019267875
60600000 106.21482468914552
60700000 106.52186339802284
60800000 90.6819222622858
60900000 47.7606865770285
61000000 109.88796808124644
61100000 102.56136841158545
61200000 102.21628333629403
61300000 107.50291588802668
61400000 95.47849385944143
61500000 96.93377568726306
61600000 90.92600467494162
61700000 28.457793995944392
61800000 96.65722760196685
61900000 96.89880671894788
62000000 98.37400504023756
62100000 100.33064978135658
62200000 102.46794792746185
62300000 100.30069720187197
62400000 104.33432890821203
62500000 33.851660620754124
62600000 112.

89100000 91.05541851130907
89200000 87.30656271862664
89300000 64.6312806394597
89400000 51.08414626825587
89500000 88.2684277746075
89600000 92.91368964884919
89700000 86.35875448508342
89800000 85.64830959174617
89900000 73.87022164081279
90000000 55.75094537021367
90100000 91.48027251645776
90200000 91.4277750208709
90300000 99.30042218963584
90400000 96.42445510082955
90500000 102.59832530046833
90600000 93.43775612660838
90700000 102.6923193161358
90800000 103.84773180591671
90900000 105.14412893942443
91000000 98.52855241023272
91100000 101.50609270736317
91200000 90.39454018269545
91300000 41.02757136975537
91400000 89.50530719692567
91500000 88.66315580121446
91600000 96.25630726553584
91700000 92.63274888507493
91800000 77.2689763652072
91900000 41.31231486405232
92000000 91.13277393819473
92100000 101.68883478218979
92200000 92.6100960517755
92300000 93.63491137188434
92400000 75.82920423515999
92500000 65.06095180407357
92600000 99.9800038340751
92700000 98.09145346176506
92

118700000 95.43747951487163
118800000 93.19174133817893
118900000 92.1280048349353
119000000 92.88366706857653
119100000 67.86177043227441
119200000 57.422446924793014
119300000 91.7873619739238
119400000 91.48476148298268
119500000 51.67099901896602
119600000 67.14400796189884
119700000 82.77857393333687
119800000 88.96349301092435
119900000 51.04287245689601
120000000 74.95660793447288
120100000 93.19307027065013
120200000 89.51134901276525
120300000 89.75017184689023
120400000 96.48974163777962
120500000 104.6315066593676
120600000 103.23194061331674
120700000 102.60293795101074
120800000 99.87923113796147
120900000 101.90958424593607
121000000 101.65557113106915
121100000 102.20794036194823
121200000 104.86687079564665
121300000 91.25924016783337
121400000 94.86758417322646
121500000 92.58103341990861
121600000 88.52790259672426
121700000 31.683163000516508
121800000 96.22831654606954
121900000 92.10462566193992
122000000 67.40241309992194
122100000 53.85883611134992
122200000 89.2

...

952600000 1.3096026416436752
952700000 1.3140126160051213
952800000 0.6092208739132544
952900000 1.4582318327619281
953000000 1.3359323160294394
953100000 1.7935561549021226
953200000 1.302557057309262
953300000 2.056999642341472
953400000 0.6899323620007753
953500000 0.7641929175746275
953600000 2.444393882114468
953700000 2.7337737870702763
953800000 1.4225389071278731
953900000 1.5441625793940026
954000000 0.9465784022122206
954100000 0.5939813401935637
954200000 1.7153986095963147
954300000 1.7249903081736624
954400000 1.43045696563261
954500000 1.2901604717775543
954600000 1.1767961157875937
954700000 0.5241976394133582
954800000 1.173432859406772
954900000 1.3684132691537716
best so far: 0
type: [1, 2, 1, 17, 4] 136
cases of this type: 2515456
955000000 1.526109983394297
955100000 2.001997018048057
955200000 1.9780229520526051
955300000 1.5999835075142628
955400000 1.1120654943570143
955500000 1.1377359994803844
955600000 1.0570633349115623
955700000 1.3981631033483293
955800000 

best so far: 0
type: [1, 2, 4, 17, 1] 136
cases of this type: 157216
979100000 0.47326215624528195
best so far: 0
type: [1, 2, 17, 1, 4] 136
cases of this type: 147968
979200000 0.31561816428432665
979300000 0.46959352798296544
best so far: 0
type: [1, 2, 17, 2, 2] 136
cases of this type: 73984
979400000 0.33044530560252466
best so far: 0
type: [1, 2, 17, 4, 1] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 1, 2] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 2, 1] 136
cases of this type: 18496
best so far: 0
type: [1, 2, 68, 1, 1] 136
cases of this type: 9248
979500000 0.30853908197512625
best so far: 0
type: [1, 4, 1, 1, 34] 136
cases of this type: 5345344
979600000 2.579293621679686
979700000 1.8012635071683984
979800000 1.64251584568758
979900000 1.2498138662061182
980000000 1.677953929658758
980100000 1.4774628969267645
980200000 1.507692080139795
980300000 1.805661421928841
980400000 1.2622949280460656
980500000 1.2848790979303406
980600000 1.1161647

998100000 0.8522709134120107
998200000 1.4952490994244607
998300000 1.3410949141861837
998400000 0.8410252607151986
998500000 1.8116132032820007
998600000 1.3510083661243948
998700000 0.8965233121967118
998800000 1.1284978948694806
998900000 1.3570694115497788
999000000 1.3679176717582007
999100000 1.7588118535306037
999200000 1.7115063717386283
999300000 1.7987540194296898
999400000 1.0002592485331563
999500000 1.7353293675306989
999600000 1.3949807118904232
999700000 0.9977672872369303
999800000 1.8061761814637471
999900000 1.3246878910137874
1000000000 0.7739726755666626
1000100000 1.5704852644795968
1000200000 1.6534542840169608
1000300000 0.9064405474568831
1000400000 1.4020668174437971
1000500000 0.2413244211489598
1000600000 0.24122240083880356
1000700000 0.24187836379906072
1000800000 0.24131153208364844
1000900000 0.2389767909543024
1001000000 0.24024059521844784
1001100000 0.3921930289206743
1001200000 0.23675440993887653
1001300000 0.23521586837943526
1001400000 0.2351676424

1025000000 0.3285203320467843
1025100000 0.21546717191762724
1025200000 0.26877115288323494
1025300000 0.4997765627500694
1025400000 0.2503856894035995
1025500000 0.2542163449688766
1025600000 0.3392677700801055
1025700000 0.4772550897084018
1025800000 0.23023257876179815
1025900000 0.3065903192109823
1026000000 0.2773958028936445
1026100000 0.19289946518575415
1026200000 0.3022258115621984
1026300000 0.2879799776155293
1026400000 0.15638379150585732
1026500000 0.3026891627185742
1026600000 0.28332374603717725
1026700000 0.16253022814971804
1026800000 0.2980579109172669
1026900000 0.27487475634629926
1027000000 0.19336818591194813
1027100000 0.2809936453888999
1027200000 0.2833188788540125
1027300000 0.20075617792607878
1027400000 0.25732200327557325
1027500000 0.31247995660990874
1027600000 0.21946200035637747
1027700000 0.22690491085343292
best so far: 0
type: [2, 1, 1, 17, 4] 136
cases of this type: 1257728
1027800000 0.29806027614055475
1027900000 0.2551387426144521
1028000000 0.23

1045400000 0.040428929289813835
1045500000 0.05271559043035705
1045600000 0.0463662477695677
1045700000 0.04933356136540837
1045800000 0.04510950688157082
1045900000 0.03530804889567097
1046000000 0.027546744348343876
1046100000 0.010112213297724725
1046200000 0.029479046285819346
1046300000 0.030858184327252708
1046400000 0.03275502804221313
1046500000 0.027957853085197343
1046600000 0.02638750971696443
best so far: 0
type: [4, 1, 1, 2, 17] 136
cases of this type: 668168
1046700000 0.02745460454554359
1046800000 0.015494677962983316
1046900000 0.013443198454131021
1047000000 0.010534103025114537
1047100000 0.005111036846372154
1047200000 0.008437151298163335
1047300000 0.009069537334084513
best so far: 0
type: [4, 1, 1, 17, 2] 136
cases of this type: 78608
1047400000 0.00602694690630436
best so far: 0
type: [4, 1, 1, 34, 1] 136
cases of this type: 39304
best so far: 0
type: [4, 1, 2, 1, 17] 136
cases of this type: 334084
1047500000 0.003667908123226961
1047600000 0.005330775370923678
